In [2]:
import numpy as np
import os 
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import models, transforms, datasets

import matplotlib.pyplot as plt
from PIL import Image

In [3]:
def pad2(img, target_length=224):
    img_height = img.shape[0]
    img_width = img.shape[1]
    img_target = np.zeros((target_length, target_length, 3), np.uint8) #<-- use this to build datase
    
    copy_loc_h = (target_length-img_height) // 2
    copy_loc_w = (target_length-img_width) // 2
    img_target[copy_loc_h:copy_loc_h+img_height, copy_loc_w:copy_loc_w+img_width,:] = img

    return img_target

# Normalized Data

In [4]:
data_transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#dataset = datasets.ImageFolder(root=data_dir, transform=data_transform)
#trainset, valset = torch.utils.data.random_split(dataset, [len(dataset)-150, 150])

trainset1 = datasets.ImageFolder(root="./Downloads/Sat2/Resolutions/Data-MS-Pad-NoCS/Train", transform=data_transform)
valset1 = datasets.ImageFolder(root="./Downloads/Sat2/Resolutions/Data-MS-Pad-NoCS/Val", transform=data_transform)

train_dl1 = torch.utils.data.DataLoader(trainset1, batch_size=64, shuffle=True, num_workers=0)
val_dl1 = torch.utils.data.DataLoader(valset1, batch_size=64, shuffle=True, num_workers=0)

In [97]:
model1 = models.resnet34(pretrained=False)
#model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 6)
model1.fc = torch.nn.Sequential(
    torch.nn.Linear(model1.fc.in_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(256, 6)
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model1.to(device)
print()

In [98]:
model1.load_state_dict(torch.load("./Downloads/Sat2/Models/Phase4_res34_fc512_256_6.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [104]:
model1.eval()
with torch.no_grad():
        val_correct_count = 0.0
        for images, labels in iter(val_dl1):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model1(images)
            predictions = outputs.argmax(dim=1)
            val_correct_count += float(torch.sum(predictions==labels))

val_accuracy = float(val_correct_count / len(valset1))

print(f"val accuracy {val_accuracy}")

val accuracy 0.9796772265391512


In [ ]:
nb_classes = 6

confusion_matrix = torch.zeros(nb_classes, nb_classes)
model1.eval()
with torch.no_grad():
    for images, labels in iter(val_dl1):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(labels.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                
confusion_matrix = confusion_matrix.numpy()
print(confusion_matrix)

In [ ]:
import pandas as pd
import seaborn as sn

classes = valset.classes

df_cm = pd.DataFrame(confusion_matrix, classes, classes)
plt.figure(figsize = (10,10))
sn.heatmap(df_cm, annot=True)

# Not Normalized Data

In [95]:
model2 = models.resnet34(pretrained=False)
#model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 6)
model2.fc = torch.nn.Sequential(
    torch.nn.Linear(model2.fc.in_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.5),
    torch.nn.Linear(256, 6)
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model2.to(device)
print()

In [96]:
model2.load_state_dict(torch.load("./Downloads/Sat2/Models/Phase4_res34_fc512_256_6_unnormalized.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [14]:
data_transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
])

#dataset = datasets.ImageFolder(root=data_dir, transform=data_transform)
#trainset, valset = torch.utils.data.random_split(dataset, [len(dataset)-150, 150])

trainset = datasets.ImageFolder(root="./Downloads/Sat2/Resolutions/Data-MS-Pad-NoCS/Train", transform=data_transform)
valset = datasets.ImageFolder(root="./Downloads/Sat2/Resolutions/Data-MS-Pad-NoCS/Val", transform=data_transform)

train_dl = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)
val_dl = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True, num_workers=0)

In [15]:
model2.eval()
with torch.no_grad():
        val_correct_count = 0.0
        for images, labels in iter(val_dl):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model2(images)
            predictions = outputs.argmax(dim=1)
            val_correct_count += float(torch.sum(predictions==labels))

val_accuracy = float(val_correct_count / len(valset))

print(f"val accuracy {val_accuracy}")

val accuracy 0.9567244471010161


# Ensemble

In [101]:
"""
LOAD MODEL GIVEN TRAINED WEIGHTS
gpu=True if GPU is available
"""
def load_model(model_dir, gpu=True):
    model = models.resnet34(pretrained=False)
    #model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 6)
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(model.fc.in_features, 256),
        torch.nn.ReLU(),
        torch.nn.Dropout(p=0.5),
        torch.nn.Linear(256, 6)
    )

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    if(gpu):
        mode.load_state_dict(torch.load(model_dir))
    else:
        model.load_state_dict(torch.load(model_dir, map_location=torch.device('cpu')))
    
    return model


"""
TEST ACCURACY OF THE ENSEMBLED MODEL WITH DIFFERENT PARAMETERS
data_dir - path to testing dataset
model1 - the first classifier
model2 - the second classifier
alpha - weight of model1
beta - weight of model2

output = alpha * model1  + beta * model2
"""
def ensemble_test(data_dir, model1, model2, alpha, beta):
    model1.eval()
    model2.eval()
    
    data_transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    valset = datasets.ImageFolder(root=data_dir, transform=data_transform)
    val_dl = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True, num_workers=0)

    with torch.no_grad():
            val_correct_count = 0.0
            for images, labels in iter(val_dl):
                images = images.to(device)
                labels = labels.to(device)

                out2 = model2(images) # output from Un-normalized input model
                
                """
                imgs = torch.zeros(images.shape[0],3,224,224)
                i=0 
                for img in images:
                    img = img.permute(1,2,0)
                    img = (img - torch.Tensor([0.485, 0.456, 0.406])) / torch.Tensor([0.229, 0.224, 0.225])
                    img = img.permute(2,0,1)
                    imgs[i] = img
                    i+=1
                """
                
                images = images.permute(0,2,3,1)
                images = (images - torch.Tensor([0.485, 0.456, 0.406])) / torch.Tensor([0.229, 0.224, 0.225])
                images = images.permute(0,3,1,2)
    
                out1 = model1(images) # output from Normalized input model

                outputs = alpha * out1 + beta * out2 # ENSEMBLING
                predictions = outputs.argmax(dim=1)
                val_correct_count += float(torch.sum(predictions==labels))

                print(val_correct_count)

    val_accuracy = float(val_correct_count / len(valset))

    print(f"val accuracy {val_accuracy}")
    
    return val_accuracy

In [102]:
model1 = load_model(model_dir = "./Downloads/Sat2/Models/Phase4_res34_fc512_256_6.pth", gpu=False)
model2 = load_model(model_dir = "./Downloads/Sat2/Models/Phase4_res34_fc512_256_6_unnormalized.pth", gpu=False)
val_dl = load_test_set(data_dir = "./Downloads/Sat2/Resolutions/Data-MS-Pad-NoCS/Val")

In [108]:
ensemble_test(data_dir="./Downloads/Sat2/Resolutions/Data-MS-Pad-NoCS/Val", 
              model1=model1, model2=model2, alpha=0.8, beta=0.2)

63.0
126.0
189.0
251.0
314.0
378.0
440.0
497.0
560.0
624.0
688.0
752.0
816.0
880.0
943.0
1005.0
1069.0
1128.0
1189.0
1253.0
1315.0
1377.0
1438.0
1501.0
1564.0
1624.0
1687.0
1751.0
1815.0
1879.0
1942.0
2005.0
2069.0
2128.0
2191.0
2253.0
2315.0
2378.0
2441.0
2505.0
2569.0
2633.0
2694.0
2757.0
2819.0
2882.0
2944.0
3007.0
3070.0
3134.0
3196.0
3259.0
3323.0
3384.0
3448.0
3510.0
3573.0
3637.0
3701.0
3764.0
3826.0
3889.0
3950.0
4014.0
4076.0
4140.0
4204.0
4267.0
4329.0
4393.0
4456.0
4518.0
4580.0
4643.0
4706.0
4768.0
4831.0
4893.0
4957.0
5019.0
5081.0
5144.0
5206.0
5268.0
5331.0
5394.0
5458.0
5522.0
5585.0
5648.0
5712.0
5775.0
5839.0
5903.0
5965.0
6028.0
6092.0
6154.0
6215.0
6278.0
6342.0
6405.0
6468.0
6531.0
6594.0
6657.0
6720.0
6783.0
6846.0
6909.0
6972.0
7036.0
7098.0
7161.0
7223.0
7287.0
7350.0
7413.0
7477.0
7541.0
7604.0
7668.0
7732.0
7796.0
7860.0
7923.0
7986.0
8047.0
8110.0
8174.0
8218.0
val accuracy 0.9824267782426779


0.9824267782426779

# Coarse-to-fine Alpha and Beta

In [113]:
def coarse():
    for alpha in np.arange(1.0,0.0,-0.1):
        beta = 1.0-alpha
        print(alpha, beta)
        

1.0 0.0
0.9 0.09999999999999998
0.8 0.19999999999999996
0.7000000000000001 0.29999999999999993
0.6000000000000001 0.3999999999999999
0.5000000000000001 0.4999999999999999
0.40000000000000013 0.5999999999999999
0.30000000000000016 0.6999999999999998
0.20000000000000018 0.7999999999999998
0.1000000000000002 0.8999999999999998


In [ ]:
imgs = torch.Tensor(64,3,224,224)
img = torch.Tensor(3,224,224)
imgs[1] = img

In [58]:
torch.Tensor([0.485, 0.456, 0.406])

tensor([0.4850, 0.4560, 0.4060])